Resolution modification

In [47]:
from PIL import Image,ImageEnhance
from dotenv import load_dotenv 
import boto3
import os
import numpy as np
import math

In [81]:

dotenv_path = os.path.join("..",'.env')
load_dotenv(dotenv_path)

#Connect to AWS Ressource, Credentials are optionals as we are accessing from the same AWS user
client=boto3.client('rekognition',aws_access_key_id=os.getenv("AWS_ACCESS_KEY_ID"),
    aws_secret_access_key=os.getenv("AWS_SECRET_ACCESS_KEY"),
    region_name=os.getenv("REGION_NAME"))
s3 = boto3.client('s3',
    aws_access_key_id=os.getenv("AWS_ACCESS_KEY_ID"),
    aws_secret_access_key=os.getenv("AWS_SECRET_ACCESS_KEY"),
    region_name=os.getenv("REGION_NAME")
)

#Define bucket name to access
bucket_name = os.getenv("BUCKET_NAME")

#Folder of real classroom photos
classroomFolder=r"C:\Users\User\Desktop\FinalProject\ImagesRekognition\RealClassroomPhotos"

In [32]:

def calculate_iou(box1, box2):

    name1,x1, y1, w1, h1 = box1
    name2, x2, y2, w2, h2 = box2
    
    x_intersect_1 = max(x1, x2)
    y_intersect_1 = max(y1, y2)
    x_intersect_2 = min(x1 + w1, x2 + w2)
    y_intersect_2 = min(y1 + h1, y2 + h2)
    
    if x_intersect_2 <= x_intersect_1 or y_intersect_2 <= y_intersect_1:
        return 0.0
    
    area_intersect = (x_intersect_2 - x_intersect_1) * (y_intersect_2 - y_intersect_1)
    area_box1 = w1 * h1
    area_box2 = w2 * h2
    
    area_union = area_box1 + area_box2 - area_intersect
    
    iou = area_intersect / area_union
    
    return iou


In [14]:
true=True
faceGalleryArray=[
{"classroom1.jpeg156424":{"filename":"classroom1.jpeg","size":156424,"regions":[{"shape_attributes":{"name":"rect","x":910,"y":348,"width":21,"height":27},"region_attributes":{"name":"AlbaQuiroz","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":733,"y":402,"width":29,"height":34},"region_attributes":{"name":"AlejandroCasallas","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":344,"y":485,"width":38,"height":44},"region_attributes":{"name":"AlejandroVelasco","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":612,"y":337,"width":20,"height":23},"region_attributes":{"name":"AlexanderDominguez","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":164,"y":481,"width":35,"height":38},"region_attributes":{"name":"AndresAnillo","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":567,"y":548,"width":57,"height":58},"region_attributes":{"name":"CarlosBornacelly","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":403,"y":582,"width":59,"height":78},"region_attributes":{"name":"DavidChaparro","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":693,"y":332,"width":20,"height":22},"region_attributes":{"name":"EstebanToro","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":15,"y":506,"width":35,"height":40},"region_attributes":{"name":"GabrielCastanez","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":1189,"y":330,"width":31,"height":31},"region_attributes":{"name":"GiohanOlivares","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":122,"y":538,"width":38,"height":55},"region_attributes":{"name":"JesusCotes","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":440,"y":368,"width":24,"height":26},"region_attributes":{"name":"JoseVilla","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":806,"y":460,"width":47,"height":55},"region_attributes":{"name":"ManuelRangel","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":1158,"y":366,"width":26,"height":33},"region_attributes":{"name":"MariaBerdugo","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":1023,"y":399,"width":30,"height":37},"region_attributes":{"name":"MariaLopez","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":717,"y":350,"width":22,"height":27},"region_attributes":{"name":"MauricioDeLaHoz","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":547,"y":431,"width":37,"height":44},"region_attributes":{"name":"SergioFonseca","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":372,"y":427,"width":28,"height":33},"region_attributes":{"name":"YordiRochel","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":616,"y":374,"width":30,"height":37},"region_attributes":{"name":"JuanMolina","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}}],"file_attributes":{"caption":"","public_domain":"no","image_url":""}}}
,{"classroom2.jpeg152579":{"filename":"classroom2.jpeg","size":152579,"regions":[{"shape_attributes":{"name":"rect","x":910,"y":347,"width":25,"height":28},"region_attributes":{"name":"AlbaQuiroz","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":729,"y":394,"width":31,"height":35},"region_attributes":{"name":"AlejandroCasallas","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":340,"y":481,"width":37,"height":46},"region_attributes":{"name":"AlejandroVelasco","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":617,"y":336,"width":20,"height":25},"region_attributes":{"name":"AlexanderDominguez","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":160,"y":481,"width":31,"height":37},"region_attributes":{"name":"AndresAnillo","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":580,"y":541,"width":55,"height":65},"region_attributes":{"name":"CarlosBornacelly","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":428,"y":580,"width":51,"height":76},"region_attributes":{"name":"DavidChaparro","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":693,"y":333,"width":20,"height":20},"region_attributes":{"name":"EstebanToro","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":28,"y":504,"width":29,"height":38},"region_attributes":{"name":"GabrielCastanez","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":1185,"y":328,"width":27,"height":32},"region_attributes":{"name":"GiohanOlivares","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":171,"y":538,"width":40,"height":57},"region_attributes":{"name":"JesusCotes","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":443,"y":366,"width":23,"height":27},"region_attributes":{"name":"JoseVilla","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":809,"y":459,"width":46,"height":53},"region_attributes":{"name":"ManuelRangel","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":1126,"y":375,"width":25,"height":33},"region_attributes":{"name":"MariaBerdugo","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":1039,"y":395,"width":25,"height":36},"region_attributes":{"name":"MariaLopez","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":720,"y":350,"width":20,"height":30},"region_attributes":{"name":"MauricioDeLaHoz","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":542,"y":428,"width":38,"height":43},"region_attributes":{"name":"SergioFonseca","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":373,"y":425,"width":27,"height":32},"region_attributes":{"name":"YordiRochel","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":613,"y":370,"width":35,"height":39},"region_attributes":{"name":"JuanMolina","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}}],"file_attributes":{"caption":"","public_domain":"no","image_url":""}}}
,{"classroom3.jpg155001":{"filename":"classroom3.jpg","size":155001,"regions":[{"shape_attributes":{"name":"rect","x":526,"y":587,"width":59,"height":70},"region_attributes":{"name":"DanielDelgado","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":605,"y":499,"width":37,"height":47},"region_attributes":{"name":"DiegoGomez","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":771,"y":537,"width":43,"height":51},"region_attributes":{"name":"WilmanDaza","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":795,"y":452,"width":31,"height":39},"region_attributes":{"name":"FelipeOspino","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":1156,"y":408,"width":23,"height":32},"region_attributes":{"name":"HabidAbdala","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":1107,"y":430,"width":26,"height":32},"region_attributes":{"name":"AngieValencia","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":1026,"y":479,"width":31,"height":37},"region_attributes":{"name":"AllysonSalom","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":356,"y":707,"width":66,"height":80},"region_attributes":{"name":"DylanDeOro","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":60,"y":790,"width":76,"height":90},"region_attributes":{"name":"JuanLozano","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}}],"file_attributes":{"caption":"","public_domain":"no","image_url":""}}}
,{"classroom4.jpg156291":{"filename":"classroom4.jpg","size":156291,"regions":[{"shape_attributes":{"name":"rect","x":552,"y":592,"width":48,"height":62},"region_attributes":{"name":"DanielDelgado","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":609,"y":500,"width":35,"height":49},"region_attributes":{"name":"DiegoGomez","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":770,"y":538,"width":39,"height":51},"region_attributes":{"name":"WilmanDaza","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":795,"y":453,"width":28,"height":39},"region_attributes":{"name":"FelipeOspino","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":1023,"y":480,"width":31,"height":37},"region_attributes":{"name":"AllysonSalom","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":1108,"y":431,"width":25,"height":33},"region_attributes":{"name":"AngieValencia","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":1155,"y":410,"width":25,"height":30},"region_attributes":{"name":"HabidAbdala","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}}],"file_attributes":{"caption":"","public_domain":"no","image_url":""}}}
,{"classroom5.jpg156524":{"filename":"classroom5.jpg","size":156524,"regions":[{"shape_attributes":{"name":"rect","x":551,"y":592,"width":49,"height":59},"region_attributes":{"name":"DanielDelgado","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":769,"y":540,"width":42,"height":49},"region_attributes":{"name":"WilmanDaza","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":607,"y":500,"width":36,"height":45},"region_attributes":{"name":"DiegoGomez","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":795,"y":452,"width":29,"height":41},"region_attributes":{"name":"FelipeOspino","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":1020,"y":480,"width":33,"height":39},"region_attributes":{"name":"AllysonSalom","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":1108,"y":431,"width":26,"height":34},"region_attributes":{"name":"AngieValencia","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":1156,"y":410,"width":25,"height":30},"region_attributes":{"name":"HabidAbdala","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}}],"file_attributes":{"caption":"","public_domain":"no","image_url":""}}}
,{"classroom6.jpg153504":{"filename":"classroom6.jpg","size":153504,"regions":[{"shape_attributes":{"name":"rect","x":59,"y":845,"width":67,"height":74},"region_attributes":{"name":"MayraCarreno","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":221,"y":721,"width":58,"height":72},"region_attributes":{"name":"JuanSanchez","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":434,"y":701,"width":50,"height":60},"region_attributes":{"name":"JuanQuintero","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":584,"y":657,"width":47,"height":47},"region_attributes":{"name":"SamirBarcelo","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":772,"y":616,"width":42,"height":45},"region_attributes":{"name":"EdwinUtria","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":976,"y":539,"width":29,"height":37},"region_attributes":{"name":"OctavioMorales","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":1077,"y":516,"width":26,"height":30},"region_attributes":{"name":"MariaFerrer","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":957,"y":472,"width":25,"height":28},"region_attributes":{"name":"SteevenBallena","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":903,"y":501,"width":23,"height":29},"region_attributes":{"name":"BrayanRubiano","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":811,"y":526,"width":21,"height":29},"region_attributes":{"name":"AndreaQuintero","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":704,"y":532,"width":31,"height":34},"region_attributes":{"name":"JuanHernandez","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":574,"y":567,"width":40,"height":49},"region_attributes":{"name":"DiegoGomez","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":343,"y":557,"width":27,"height":29},"region_attributes":{"name":"AndresFabregas","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":243,"y":581,"width":30,"height":31},"region_attributes":{"name":"GabrielaBecerra","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":290,"y":520,"width":21,"height":26},"region_attributes":{"name":"AliRada","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":193,"y":553,"width":24,"height":26},"region_attributes":{"name":"LucianaDeLaRosa","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":155,"y":576,"width":31,"height":36},"region_attributes":{"name":"AndresNarvaez","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}}],"file_attributes":{"caption":"","public_domain":"no","image_url":""}}}
,{"classroom7.jpg155011":{"filename":"classroom7.jpg","size":155011,"regions":[{"shape_attributes":{"name":"rect","x":46,"y":849,"width":64,"height":79},"region_attributes":{"name":"MayraCarreno","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":219,"y":724,"width":58,"height":71},"region_attributes":{"name":"JuanSanchez","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":430,"y":696,"width":55,"height":65},"region_attributes":{"name":"JuanQuintero","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":589,"y":661,"width":46,"height":49},"region_attributes":{"name":"SamirBarcelo","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":771,"y":612,"width":44,"height":54},"region_attributes":{"name":"EdwinUtria","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":969,"y":539,"width":32,"height":39},"region_attributes":{"name":"OctavioMorales","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":1069,"y":518,"width":29,"height":33},"region_attributes":{"name":"MariaFerrer","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":954,"y":475,"width":25,"height":28},"region_attributes":{"name":"SteevenBallena","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":898,"y":501,"width":27,"height":29},"region_attributes":{"name":"BrayanRubiano","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":807,"y":527,"width":24,"height":29},"region_attributes":{"name":"AndreaQuintero","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":575,"y":569,"width":38,"height":47},"region_attributes":{"name":"DiegoGomez","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":706,"y":531,"width":31,"height":34},"region_attributes":{"name":"JuanHernandez","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":343,"y":557,"width":27,"height":31},"region_attributes":{"name":"AndresFabregas","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":290,"y":505,"width":24,"height":29},"region_attributes":{"name":"AliRada","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":244,"y":579,"width":28,"height":31},"region_attributes":{"name":"GabrielaBecerra","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":186,"y":555,"width":24,"height":26},"region_attributes":{"name":"LucianaDeLaRosa","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":134,"y":574,"width":31,"height":38},"region_attributes":{"name":"AndresNarvaez","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}}],"file_attributes":{"caption":"","public_domain":"no","image_url":""}}}
]



In [ ]:
true=True
faceGallery=[
    faceGalleryArray[5]
]
tuples=[]
m=0
for element in faceGallery:
    i=1
    filteredFaceGalleryObject=element[next(iter(element))]["regions"]
    tuplesnthPhoto=[]
    for element in filteredFaceGalleryObject:
        region=element["shape_attributes"]
        identity=element["region_attributes"]
        tuplesnthPhoto.append((identity["name"],region["x"],region["y"], region["width"], region["height"]))
        i=i+1
    tuples.append(tuplesnthPhoto)
    m=m+1


In [ ]:
student_photos_path=r"C:\Users\User\Desktop\FinalProject\ImagesRekognition\StudentsElectronicDesign2"
camera_frame=r"C:\Users\User\Desktop\FinalProject\ImagesRekognition\RealClassroomPhotos\classroom00110.jpg"
camera_photo= Image.open(camera_frame)
cameraWidth,cameraHeight=camera_photo.size

with open(camera_frame, 'rb') as image_file:
    camera_frame_bytes = image_file.read()
results=[]

i=0
for student in os.listdir(student_photos_path):
    multiplier=1
    #Define path to individual photo
    student_id_photo=rf"C:\Users\User\Desktop\FinalProject\ImagesRekognition\StudentsElectronicDesign2\{student}"
    #Obtain image's dimensions in order to know if its size is valid for the minimal rekognition's requirements
    baseImage=Image.open(student_id_photo)
    initialWidth, initialHeight=baseImage.size
    initialBox=True

    while ((initialWidth/multiplier)>80 and (initialHeight/multiplier)>80):
        #Take base image, resize and save it according to a multiplier
        finalWidth=initialWidth/multiplier
        finalHeight=initialHeight/multiplier
        student_id_photo_resized=baseImage.resize((int(finalWidth),int(finalHeight)))
        student_id_photo_resized.save(f'{student}_scaled_{finalHeight}x{finalWidth}.jpg')
        student_id_photo_resized_path=f'{student}_scaled_{finalHeight}x{finalWidth}.jpg'
        size=os.path.getsize(student_id_photo_resized_path)
        TPSimilarityAverage=0
        FPSimilarityAverage=0
        FPCounter=0
        TP=TN=FN=FP=0
        #Convert to bytes the resixed image
        with open(student_id_photo_resized_path, 'rb') as image_file:
            student_id_photo_resized_bytes = image_file.read()

        #Look for the resized students photo in the group photo, reduce threshold to obtain similarity info about all faces detected


        responseRecognition=client.compare_faces(
                    SourceImage={'Bytes': student_id_photo_resized_bytes},
                    TargetImage={'Bytes': camera_frame_bytes},
                    SimilarityThreshold=0
                )
        
        #Save the corresponding face gallery coordinates in order to compare
        if (initialBox):
            for element in tuples[0]:
                if element[0]==student.split(".")[0]:
                    trueBoundingBoxWithId=(student,element[1]/cameraWidth,element[2]/cameraHeight,element[3]/cameraWidth,element[4]/cameraHeight)
                    break

        initialBox=False

        print(f"Results for resolution {finalWidth}x{finalHeight}")

        for face in responseRecognition["FaceMatches"]:
            partialBB=face["Face"]["BoundingBox"]
            IoU=calculate_iou(trueBoundingBoxWithId, ("Name",partialBB["Left"],partialBB["Top"],partialBB["Width"],partialBB["Height"]))
            if (IoU>=0.5):
                TPSimilarityAverage=TPSimilarityAverage+float(face["Similarity"])
                if(face["Similarity"]>80):
                    TP=TP+1
                print(f"{student.split(".")[0]} recognized in {partialBB} with a similarity of {face["Similarity"]}")
            else:
                FPSimilarityAverage=FPSimilarityAverage+float(face["Similarity"])
                FPCounter=FPCounter+1
                if face["Similarity"]>80:
                    FP=FP+1
                    print(f"Error occurred, the model mismatched a face in {partialBB} with {face["Similarity"]}")
                else:
                    TN=TN+1
        if (TP==0):
            FN=1
        results.append({"Name":student.split(".")[0],"Size": size,"Width": finalWidth, "Height": finalHeight, "TP Similarity": TPSimilarityAverage,
                         "Others Similarity Average": FPSimilarityAverage/FPCounter,"TP":TP,"TN":TN,"FP": FP,"FN":FN})
        multiplier=multiplier*2


    i=i+1


In [41]:

resolutionControlResults=[
[{'Name': 'AlbaQuiroz',
  'Size': 559213,
  'Width': 2640.0,
  'Height': 1980.0,
  'TP Similarity': 87.35999298095703,
  'Others Similarity Average': 2.0486047476530076,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'AlbaQuiroz',
  'Size': 149593,
  'Width': 1320.0,
  'Height': 990.0,
  'TP Similarity': 85.79832458496094,
  'Others Similarity Average': 2.2484910283237696,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'AlbaQuiroz',
  'Size': 42314,
  'Width': 660.0,
  'Height': 495.0,
  'TP Similarity': 79.09496307373047,
  'Others Similarity Average': 2.1174419827759268,
  'TP': 0,
  'TN': 20,
  'FP': 0,
  'FN': 1},
 {'Name': 'AlbaQuiroz',
  'Size': 13328,
  'Width': 330.0,
  'Height': 247.5,
  'TP Similarity': 87.31429290771484,
  'Others Similarity Average': 2.545622690021992,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'AlbaQuiroz',
  'Size': 4808,
  'Width': 165.0,
  'Height': 123.75,
  'TP Similarity': 92.87079620361328,
  'Others Similarity Average': 2.3175505198538304,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'AlejandroCasallas',
  'Size': 841318,
  'Width': 3392.0,
  'Height': 2544.0,
  'TP Similarity': 99.6025619506836,
  'Others Similarity Average': 1.0072110334411264,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'AlejandroCasallas',
  'Size': 133681,
  'Width': 1696.0,
  'Height': 1272.0,
  'TP Similarity': 99.54573822021484,
  'Others Similarity Average': 0.9120928678661585,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'AlejandroCasallas',
  'Size': 31693,
  'Width': 848.0,
  'Height': 636.0,
  'TP Similarity': 99.56169128417969,
  'Others Similarity Average': 0.9089572820812464,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'AlejandroCasallas',
  'Size': 10143,
  'Width': 424.0,
  'Height': 318.0,
  'TP Similarity': 99.67095947265625,
  'Others Similarity Average': 1.300387545861304,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'AlejandroCasallas',
  'Size': 3872,
  'Width': 212.0,
  'Height': 159.0,
  'TP Similarity': 99.67228698730469,
  'Others Similarity Average': 1.2834486234933138,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'AlejandroVelasco',
  'Size': 901524,
  'Width': 4080.0,
  'Height': 2296.0,
  'TP Similarity': 99.83711242675781,
  'Others Similarity Average': 0.9902661461383104,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'AlejandroVelasco',
  'Size': 242471,
  'Width': 2040.0,
  'Height': 1148.0,
  'TP Similarity': 99.86778259277344,
  'Others Similarity Average': 1.0161392678506673,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'AlejandroVelasco',
  'Size': 64550,
  'Width': 1020.0,
  'Height': 574.0,
  'TP Similarity': 99.88746643066406,
  'Others Similarity Average': 1.150446755439043,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'AlejandroVelasco',
  'Size': 18536,
  'Width': 510.0,
  'Height': 287.0,
  'TP Similarity': 99.9223861694336,
  'Others Similarity Average': 2.167590771988034,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'AlejandroVelasco',
  'Size': 6099,
  'Width': 255.0,
  'Height': 143.5,
  'TP Similarity': 99.95794677734375,
  'Others Similarity Average': 2.1888977205380797,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'AlexanderDominguez',
  'Size': 356373,
  'Width': 1728.0,
  'Height': 2304.0,
  'TP Similarity': 95.1724853515625,
  'Others Similarity Average': 3.1305185072124004,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'AlexanderDominguez',
  'Size': 81786,
  'Width': 864.0,
  'Height': 1152.0,
  'TP Similarity': 94.31664276123047,
  'Others Similarity Average': 3.091881028562784,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'AlexanderDominguez',
  'Size': 21956,
  'Width': 432.0,
  'Height': 576.0,
  'TP Similarity': 94.08954620361328,
  'Others Similarity Average': 3.314782378077507,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'AlexanderDominguez',
  'Size': 8065,
  'Width': 216.0,
  'Height': 288.0,
  'TP Similarity': 96.06655883789062,
  'Others Similarity Average': 3.4550859786570074,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'AlexanderDominguez',
  'Size': 3219,
  'Width': 108.0,
  'Height': 144.0,
  'TP Similarity': 98.73424530029297,
  'Others Similarity Average': 3.808628688752651,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'AndresAnillo',
  'Size': 385853,
  'Width': 1448.0,
  'Height': 2573.0,
  'TP Similarity': 91.47819519042969,
  'Others Similarity Average': 1.8632595747709275,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'AndresAnillo',
  'Size': 84330,
  'Width': 724.0,
  'Height': 1286.5,
  'TP Similarity': 90.35116577148438,
  'Others Similarity Average': 2.04094080850482,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'AndresAnillo',
  'Size': 25136,
  'Width': 362.0,
  'Height': 643.25,
  'TP Similarity': 93.69498443603516,
  'Others Similarity Average': 2.2641327895224093,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'AndresAnillo',
  'Size': 8799,
  'Width': 181.0,
  'Height': 321.625,
  'TP Similarity': 97.75790405273438,
  'Others Similarity Average': 2.6068315468728542,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'AndresAnillo',
  'Size': 3277,
  'Width': 90.5,
  'Height': 160.8125,
  'TP Similarity': 95.64801788330078,
  'Others Similarity Average': 2.8017429761588573,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'CarlosBornacelly',
  'Size': 490634,
  'Width': 1815.0,
  'Height': 2420.0,
  'TP Similarity': 99.98979187011719,
  'Others Similarity Average': 3.5141441985964774,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'CarlosBornacelly',
  'Size': 107587,
  'Width': 907.5,
  'Height': 1210.0,
  'TP Similarity': 99.98800659179688,
  'Others Similarity Average': 3.51781745031476,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'CarlosBornacelly',
  'Size': 26174,
  'Width': 453.75,
  'Height': 605.0,
  'TP Similarity': 99.99066925048828,
  'Others Similarity Average': 3.46205857694149,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'CarlosBornacelly',
  'Size': 8070,
  'Width': 226.875,
  'Height': 302.5,
  'TP Similarity': 99.9894790649414,
  'Others Similarity Average': 3.64525213688612,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'CarlosBornacelly',
  'Size': 3174,
  'Width': 113.4375,
  'Height': 151.25,
  'TP Similarity': 99.98719787597656,
  'Others Similarity Average': 4.003021792322397,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'DavidChaparro',
  'Size': 124094,
  'Width': 960.0,
  'Height': 1280.0,
  'TP Similarity': 99.97116088867188,
  'Others Similarity Average': 1.6291546270251274,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'DavidChaparro',
  'Size': 27494,
  'Width': 480.0,
  'Height': 640.0,
  'TP Similarity': 99.98489379882812,
  'Others Similarity Average': 1.9714938316494226,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'DavidChaparro',
  'Size': 7144,
  'Width': 240.0,
  'Height': 320.0,
  'TP Similarity': 99.9925308227539,
  'Others Similarity Average': 2.248844922333956,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'DavidChaparro',
  'Size': 2875,
  'Width': 120.0,
  'Height': 160.0,
  'TP Similarity': 99.98323059082031,
  'Others Similarity Average': 2.412531017139554,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'EstebanToro',
  'Size': 132921,
  'Width': 1158.0,
  'Height': 1544.0,
  'TP Similarity': 46.55854797363281,
  'Others Similarity Average': 2.299416425824165,
  'TP': 0,
  'TN': 20,
  'FP': 0,
  'FN': 1},
 {'Name': 'EstebanToro',
  'Size': 37356,
  'Width': 579.0,
  'Height': 772.0,
  'TP Similarity': 39.42287826538086,
  'Others Similarity Average': 2.5110351592302322,
  'TP': 0,
  'TN': 20,
  'FP': 0,
  'FN': 1},
 {'Name': 'EstebanToro',
  'Size': 11320,
  'Width': 289.5,
  'Height': 386.0,
  'TP Similarity': 65.79220581054688,
  'Others Similarity Average': 3.57427338026464,
  'TP': 0,
  'TN': 20,
  'FP': 0,
  'FN': 1},
 {'Name': 'EstebanToro',
  'Size': 4194,
  'Width': 144.75,
  'Height': 193.0,
  'TP Similarity': 76.55072784423828,
  'Others Similarity Average': 5.181087900325656,
  'TP': 0,
  'TN': 20,
  'FP': 0,
  'FN': 1},
 {'Name': 'GabrielCastanez',
  'Size': 263863,
  'Width': 1560.0,
  'Height': 2104.0,
  'TP Similarity': 89.58196258544922,
  'Others Similarity Average': 0.9302847081795335,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'GabrielCastanez',
  'Size': 82366,
  'Width': 780.0,
  'Height': 1052.0,
  'TP Similarity': 92.11920166015625,
  'Others Similarity Average': 1.012617457099259,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'GabrielCastanez',
  'Size': 26927,
  'Width': 390.0,
  'Height': 526.0,
  'TP Similarity': 97.96266174316406,
  'Others Similarity Average': 1.5615053799003362,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'GabrielCastanez',
  'Size': 9237,
  'Width': 195.0,
  'Height': 263.0,
  'TP Similarity': 98.952880859375,
  'Others Similarity Average': 2.1528886657208206,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'GabrielCastanez',
  'Size': 3360,
  'Width': 97.5,
  'Height': 131.5,
  'TP Similarity': 99.88145446777344,
  'Others Similarity Average': 4.535957834869623,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'GiohanOlivares',
  'Size': 275599,
  'Width': 1448.0,
  'Height': 2573.0,
  'TP Similarity': 97.7962875366211,
  'Others Similarity Average': 3.019160672277212,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'GiohanOlivares',
  'Size': 82087,
  'Width': 724.0,
  'Height': 1286.5,
  'TP Similarity': 98.33745574951172,
  'Others Similarity Average': 3.0690283298492433,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'GiohanOlivares',
  'Size': 27831,
  'Width': 362.0,
  'Height': 643.25,
  'TP Similarity': 98.51893615722656,
  'Others Similarity Average': 3.4856067396700383,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'GiohanOlivares',
  'Size': 9736,
  'Width': 181.0,
  'Height': 321.625,
  'TP Similarity': 98.62715911865234,
  'Others Similarity Average': 4.680827696621418,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'GiohanOlivares',
  'Size': 3638,
  'Width': 90.5,
  'Height': 160.8125,
  'TP Similarity': 99.53730773925781,
  'Others Similarity Average': 6.767614617943764,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'JesusCotes',
  'Size': 83374,
  'Width': 900.0,
  'Height': 1600.0,
  'TP Similarity': 99.6865234375,
  'Others Similarity Average': 1.0130176421254873,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'JesusCotes',
  'Size': 27713,
  'Width': 450.0,
  'Height': 800.0,
  'TP Similarity': 99.92517852783203,
  'Others Similarity Average': 1.2951110493391753,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'JesusCotes',
  'Size': 9490,
  'Width': 225.0,
  'Height': 400.0,
  'TP Similarity': 99.96050262451172,
  'Others Similarity Average': 1.8662166088819503,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'JesusCotes',
  'Size': 3594,
  'Width': 112.5,
  'Height': 200.0,
  'TP Similarity': 99.95608520507812,
  'Others Similarity Average': 1.7807683352380992,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'JoseVilla',
  'Size': 399377,
  'Width': 1815.0,
  'Height': 2420.0,
  'TP Similarity': 95.7163314819336,
  'Others Similarity Average': 2.422147117368877,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'JoseVilla',
  'Size': 98948,
  'Width': 907.5,
  'Height': 1210.0,
  'TP Similarity': 95.9336929321289,
  'Others Similarity Average': 2.6645775977522135,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'JoseVilla',
  'Size': 27354,
  'Width': 453.75,
  'Height': 605.0,
  'TP Similarity': 95.5064926147461,
  'Others Similarity Average': 2.198662956990302,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'JoseVilla',
  'Size': 8947,
  'Width': 226.875,
  'Height': 302.5,
  'TP Similarity': 96.9863510131836,
  'Others Similarity Average': 2.751949927210808,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'JoseVilla',
  'Size': 3451,
  'Width': 113.4375,
  'Height': 151.25,
  'TP Similarity': 98.22427368164062,
  'Others Similarity Average': 4.293895189464092,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'JuanMolina',
  'Size': 389259,
  'Width': 2448.0,
  'Height': 2448.0,
  'TP Similarity': 92.86930084228516,
  'Others Similarity Average': 1.8919164406135678,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'JuanMolina',
  'Size': 81553,
  'Width': 1224.0,
  'Height': 1224.0,
  'TP Similarity': 92.36573791503906,
  'Others Similarity Average': 2.1196896109730003,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'JuanMolina',
  'Size': 23395,
  'Width': 612.0,
  'Height': 612.0,
  'TP Similarity': 90.91831970214844,
  'Others Similarity Average': 1.7261565553024412,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'JuanMolina',
  'Size': 8089,
  'Width': 306.0,
  'Height': 306.0,
  'TP Similarity': 93.86478424072266,
  'Others Similarity Average': 2.9194627977907657,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'JuanMolina',
  'Size': 3211,
  'Width': 153.0,
  'Height': 153.0,
  'TP Similarity': 96.26791381835938,
  'Others Similarity Average': 4.27316511310637,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'ManuelRangel',
  'Size': 32563,
  'Width': 709.0,
  'Height': 945.0,
  'TP Similarity': 99.91717529296875,
  'Others Similarity Average': 3.784017212688923,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'ManuelRangel',
  'Size': 12031,
  'Width': 354.5,
  'Height': 472.5,
  'TP Similarity': 99.90631103515625,
  'Others Similarity Average': 4.5918392889201645,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'ManuelRangel',
  'Size': 4864,
  'Width': 177.25,
  'Height': 236.25,
  'TP Similarity': 99.93641662597656,
  'Others Similarity Average': 4.499638976156712,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'ManuelRangel',
  'Size': 2237,
  'Width': 88.625,
  'Height': 118.125,
  'TP Similarity': 99.7033920288086,
  'Others Similarity Average': 6.338207038491964,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'MariaBerdugo',
  'Size': 62694,
  'Width': 682.0,
  'Height': 910.0,
  'TP Similarity': 89.5107650756836,
  'Others Similarity Average': 3.3991156943142413,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'MariaBerdugo',
  'Size': 17758,
  'Width': 341.0,
  'Height': 455.0,
  'TP Similarity': 91.7619857788086,
  'Others Similarity Average': 2.7563270412385465,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'MariaBerdugo',
  'Size': 6236,
  'Width': 170.5,
  'Height': 227.5,
  'TP Similarity': 94.19857025146484,
  'Others Similarity Average': 2.3873752169311047,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'MariaBerdugo',
  'Size': 2623,
  'Width': 85.25,
  'Height': 113.75,
  'TP Similarity': 97.87590789794922,
  'Others Similarity Average': 2.460094056650996,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'MariaLopez',
  'Size': 415432,
  'Width': 2640.0,
  'Height': 1980.0,
  'TP Similarity': 99.11638641357422,
  'Others Similarity Average': 1.2658425320871174,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'MariaLopez',
  'Size': 112863,
  'Width': 1320.0,
  'Height': 990.0,
  'TP Similarity': 99.24763488769531,
  'Others Similarity Average': 1.1369418267160654,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'MariaLopez',
  'Size': 31233,
  'Width': 660.0,
  'Height': 495.0,
  'TP Similarity': 99.19293212890625,
  'Others Similarity Average': 1.1933696173131465,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'MariaLopez',
  'Size': 10439,
  'Width': 330.0,
  'Height': 247.5,
  'TP Similarity': 99.75145721435547,
  'Others Similarity Average': 1.818659815657884,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'MariaLopez',
  'Size': 4091,
  'Width': 165.0,
  'Height': 123.75,
  'TP Similarity': 99.92547607421875,
  'Others Similarity Average': 1.598639497347176,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'MauricioDeLaHoz',
  'Size': 177074,
  'Width': 1200.0,
  'Height': 1600.0,
  'TP Similarity': 76.35456085205078,
  'Others Similarity Average': 1.3674629025161267,
  'TP': 0,
  'TN': 20,
  'FP': 0,
  'FN': 1},
 {'Name': 'MauricioDeLaHoz',
  'Size': 55242,
  'Width': 600.0,
  'Height': 800.0,
  'TP Similarity': 73.25460052490234,
  'Others Similarity Average': 1.3460139214992524,
  'TP': 0,
  'TN': 20,
  'FP': 0,
  'FN': 1},
 {'Name': 'MauricioDeLaHoz',
  'Size': 18135,
  'Width': 300.0,
  'Height': 400.0,
  'TP Similarity': 87.3102798461914,
  'Others Similarity Average': 1.8444269958883523,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'MauricioDeLaHoz',
  'Size': 6269,
  'Width': 150.0,
  'Height': 200.0,
  'TP Similarity': 83.9224853515625,
  'Others Similarity Average': 2.0386156011372805,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'SergioFonseca',
  'Size': 362436,
  'Width': 1440.0,
  'Height': 2559.0,
  'TP Similarity': 99.54402160644531,
  'Others Similarity Average': 2.3292657585814593,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'SergioFonseca',
  'Size': 94811,
  'Width': 720.0,
  'Height': 1279.5,
  'TP Similarity': 99.58061218261719,
  'Others Similarity Average': 2.091276241838932,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'SergioFonseca',
  'Size': 26284,
  'Width': 360.0,
  'Height': 639.75,
  'TP Similarity': 99.49429321289062,
  'Others Similarity Average': 1.731072480790317,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'SergioFonseca',
  'Size': 8545,
  'Width': 180.0,
  'Height': 319.875,
  'TP Similarity': 99.68608093261719,
  'Others Similarity Average': 1.9310677580535411,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'SergioFonseca',
  'Size': 3222,
  'Width': 90.0,
  'Height': 159.9375,
  'TP Similarity': 99.87846374511719,
  'Others Similarity Average': 3.0457728382200004,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'YordiRochel',
  'Size': 290168,
  'Width': 1740.0,
  'Height': 2319.0,
  'TP Similarity': 98.94715881347656,
  'Others Similarity Average': 1.1727693067863583,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'YordiRochel',
  'Size': 80430,
  'Width': 870.0,
  'Height': 1159.5,
  'TP Similarity': 98.78468322753906,
  'Others Similarity Average': 1.1230029579252006,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'YordiRochel',
  'Size': 22017,
  'Width': 435.0,
  'Height': 579.75,
  'TP Similarity': 98.7977294921875,
  'Others Similarity Average': 1.3413120444864035,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'YordiRochel',
  'Size': 7461,
  'Width': 217.5,
  'Height': 289.875,
  'TP Similarity': 98.82576751708984,
  'Others Similarity Average': 1.7710456933826209,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'YordiRochel',
  'Size': 2889,
  'Width': 108.75,
  'Height': 144.9375,
  'TP Similarity': 99.09185028076172,
  'Others Similarity Average': 1.797189749777317,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0}]
 
 ,
[{'Name': 'AlbaQuiroz',
  'Size': 559213,
  'Width': 2640.0,
  'Height': 1980.0,
  'TP Similarity': 79.85643768310547,
  'Others Similarity Average': 0.9912954699248075,
  'TP': 0,
  'TN': 20,
  'FP': 0,
  'FN': 1},
 {'Name': 'AlbaQuiroz',
  'Size': 149593,
  'Width': 1320.0,
  'Height': 990.0,
  'TP Similarity': 79.59484100341797,
  'Others Similarity Average': 1.0935674071311952,
  'TP': 0,
  'TN': 20,
  'FP': 0,
  'FN': 1},
 {'Name': 'AlbaQuiroz',
  'Size': 42314,
  'Width': 660.0,
  'Height': 495.0,
  'TP Similarity': 71.27705383300781,
  'Others Similarity Average': 1.028725741058588,
  'TP': 0,
  'TN': 20,
  'FP': 0,
  'FN': 1},
 {'Name': 'AlbaQuiroz',
  'Size': 13328,
  'Width': 330.0,
  'Height': 247.5,
  'TP Similarity': 84.0685043334961,
  'Others Similarity Average': 1.2297687675803899,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'AlbaQuiroz',
  'Size': 4808,
  'Width': 165.0,
  'Height': 123.75,
  'TP Similarity': 93.41893005371094,
  'Others Similarity Average': 1.2866859935224055,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'AlejandroCasallas',
  'Size': 841318,
  'Width': 3392.0,
  'Height': 2544.0,
  'TP Similarity': 85.80200958251953,
  'Others Similarity Average': 1.1137323319911956,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'AlejandroCasallas',
  'Size': 133681,
  'Width': 1696.0,
  'Height': 1272.0,
  'TP Similarity': 84.13854217529297,
  'Others Similarity Average': 1.036678758263588,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'AlejandroCasallas',
  'Size': 31693,
  'Width': 848.0,
  'Height': 636.0,
  'TP Similarity': 83.09861755371094,
  'Others Similarity Average': 1.0214371133595705,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'AlejandroCasallas',
  'Size': 10143,
  'Width': 424.0,
  'Height': 318.0,
  'TP Similarity': 89.91259002685547,
  'Others Similarity Average': 1.4592885233461856,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'AlejandroCasallas',
  'Size': 3872,
  'Width': 212.0,
  'Height': 159.0,
  'TP Similarity': 89.73654174804688,
  'Others Similarity Average': 1.5535706598311663,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'AlejandroVelasco',
  'Size': 901524,
  'Width': 4080.0,
  'Height': 2296.0,
  'TP Similarity': 99.7657699584961,
  'Others Similarity Average': 0.9778900315985084,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'AlejandroVelasco',
  'Size': 242471,
  'Width': 2040.0,
  'Height': 1148.0,
  'TP Similarity': 99.80254364013672,
  'Others Similarity Average': 1.0036839054897428,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'AlejandroVelasco',
  'Size': 64550,
  'Width': 1020.0,
  'Height': 574.0,
  'TP Similarity': 99.84292602539062,
  'Others Similarity Average': 1.0576139032840728,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'AlejandroVelasco',
  'Size': 18536,
  'Width': 510.0,
  'Height': 287.0,
  'TP Similarity': 99.8988037109375,
  'Others Similarity Average': 1.8625946674495935,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'AlejandroVelasco',
  'Size': 6099,
  'Width': 255.0,
  'Height': 143.5,
  'TP Similarity': 99.96870422363281,
  'Others Similarity Average': 2.2063967723399402,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'AlexanderDominguez',
  'Size': 356373,
  'Width': 1728.0,
  'Height': 2304.0,
  'TP Similarity': 85.96520233154297,
  'Others Similarity Average': 3.078850771673024,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'AlexanderDominguez',
  'Size': 81786,
  'Width': 864.0,
  'Height': 1152.0,
  'TP Similarity': 84.55677795410156,
  'Others Similarity Average': 3.0272230595350265,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'AlexanderDominguez',
  'Size': 21956,
  'Width': 432.0,
  'Height': 576.0,
  'TP Similarity': 84.85446166992188,
  'Others Similarity Average': 3.091536344587803,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'AlexanderDominguez',
  'Size': 8065,
  'Width': 216.0,
  'Height': 288.0,
  'TP Similarity': 91.63568115234375,
  'Others Similarity Average': 3.3160484161227943,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'AlexanderDominguez',
  'Size': 3219,
  'Width': 108.0,
  'Height': 144.0,
  'TP Similarity': 96.78227996826172,
  'Others Similarity Average': 3.410281080752611,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'AndresAnillo',
  'Size': 385853,
  'Width': 1448.0,
  'Height': 2573.0,
  'TP Similarity': 91.48632049560547,
  'Others Similarity Average': 2.29701404646039,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'AndresAnillo',
  'Size': 84330,
  'Width': 724.0,
  'Height': 1286.5,
  'TP Similarity': 91.71785736083984,
  'Others Similarity Average': 2.32988813072443,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'AndresAnillo',
  'Size': 25136,
  'Width': 362.0,
  'Height': 643.25,
  'TP Similarity': 93.9027328491211,
  'Others Similarity Average': 2.8223344027996062,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'AndresAnillo',
  'Size': 8799,
  'Width': 181.0,
  'Height': 321.625,
  'TP Similarity': 98.1513900756836,
  'Others Similarity Average': 3.061068394780159,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'AndresAnillo',
  'Size': 3277,
  'Width': 90.5,
  'Height': 160.8125,
  'TP Similarity': 95.71681213378906,
  'Others Similarity Average': 2.5212879993021486,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'CarlosBornacelly',
  'Size': 490634,
  'Width': 1815.0,
  'Height': 2420.0,
  'TP Similarity': 99.98246765136719,
  'Others Similarity Average': 2.7596656568348408,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'CarlosBornacelly',
  'Size': 107587,
  'Width': 907.5,
  'Height': 1210.0,
  'TP Similarity': 99.98371124267578,
  'Others Similarity Average': 2.9646586500108243,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'CarlosBornacelly',
  'Size': 26174,
  'Width': 453.75,
  'Height': 605.0,
  'TP Similarity': 99.98431396484375,
  'Others Similarity Average': 2.8735698498785496,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'CarlosBornacelly',
  'Size': 8070,
  'Width': 226.875,
  'Height': 302.5,
  'TP Similarity': 99.98466491699219,
  'Others Similarity Average': 3.208473153412342,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'CarlosBornacelly',
  'Size': 3174,
  'Width': 113.4375,
  'Height': 151.25,
  'TP Similarity': 99.98111724853516,
  'Others Similarity Average': 2.963789143785834,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'DavidChaparro',
  'Size': 124094,
  'Width': 960.0,
  'Height': 1280.0,
  'TP Similarity': 99.9983901977539,
  'Others Similarity Average': 2.5572612468153237,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'DavidChaparro',
  'Size': 27494,
  'Width': 480.0,
  'Height': 640.0,
  'TP Similarity': 99.99907684326172,
  'Others Similarity Average': 2.850088299438357,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'DavidChaparro',
  'Size': 7144,
  'Width': 240.0,
  'Height': 320.0,
  'TP Similarity': 99.99913024902344,
  'Others Similarity Average': 2.8359414167702197,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'DavidChaparro',
  'Size': 2875,
  'Width': 120.0,
  'Height': 160.0,
  'TP Similarity': 99.99905395507812,
  'Others Similarity Average': 3.0160314194858073,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'EstebanToro',
  'Size': 132921,
  'Width': 1158.0,
  'Height': 1544.0,
  'TP Similarity': 24.16159439086914,
  'Others Similarity Average': 1.6524500973522662,
  'TP': 0,
  'TN': 20,
  'FP': 0,
  'FN': 1},
 {'Name': 'EstebanToro',
  'Size': 37356,
  'Width': 579.0,
  'Height': 772.0,
  'TP Similarity': 21.789512634277344,
  'Others Similarity Average': 1.7996781155467034,
  'TP': 0,
  'TN': 20,
  'FP': 0,
  'FN': 1},
 {'Name': 'EstebanToro',
  'Size': 11320,
  'Width': 289.5,
  'Height': 386.0,
  'TP Similarity': 34.01555252075195,
  'Others Similarity Average': 2.3668348670005797,
  'TP': 0,
  'TN': 20,
  'FP': 0,
  'FN': 1},
 {'Name': 'EstebanToro',
  'Size': 4194,
  'Width': 144.75,
  'Height': 193.0,
  'TP Similarity': 49.52687454223633,
  'Others Similarity Average': 4.028890110552311,
  'TP': 0,
  'TN': 20,
  'FP': 0,
  'FN': 1},
 {'Name': 'GabrielCastanez',
  'Size': 263863,
  'Width': 1560.0,
  'Height': 2104.0,
  'TP Similarity': 88.95003509521484,
  'Others Similarity Average': 0.673047487437725,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'GabrielCastanez',
  'Size': 82366,
  'Width': 780.0,
  'Height': 1052.0,
  'TP Similarity': 89.80128479003906,
  'Others Similarity Average': 0.7366241917014122,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'GabrielCastanez',
  'Size': 26927,
  'Width': 390.0,
  'Height': 526.0,
  'TP Similarity': 97.87480163574219,
  'Others Similarity Average': 1.3414684895426034,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'GabrielCastanez',
  'Size': 9237,
  'Width': 195.0,
  'Height': 263.0,
  'TP Similarity': 98.12397003173828,
  'Others Similarity Average': 1.5155781641602517,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'GabrielCastanez',
  'Size': 3360,
  'Width': 97.5,
  'Height': 131.5,
  'TP Similarity': 99.45720672607422,
  'Others Similarity Average': 2.9710221961140633,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'GiohanOlivares',
  'Size': 275599,
  'Width': 1448.0,
  'Height': 2573.0,
  'TP Similarity': 93.43003845214844,
  'Others Similarity Average': 3.189732679724693,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'GiohanOlivares',
  'Size': 82087,
  'Width': 724.0,
  'Height': 1286.5,
  'TP Similarity': 94.18968200683594,
  'Others Similarity Average': 3.4133755169808864,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'GiohanOlivares',
  'Size': 27831,
  'Width': 362.0,
  'Height': 643.25,
  'TP Similarity': 95.83951568603516,
  'Others Similarity Average': 3.9451188549399374,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'GiohanOlivares',
  'Size': 9736,
  'Width': 181.0,
  'Height': 321.625,
  'TP Similarity': 97.74807739257812,
  'Others Similarity Average': 4.618668334931135,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'GiohanOlivares',
  'Size': 3638,
  'Width': 90.5,
  'Height': 160.8125,
  'TP Similarity': 99.6102294921875,
  'Others Similarity Average': 6.693490266799927,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'JesusCotes',
  'Size': 83374,
  'Width': 900.0,
  'Height': 1600.0,
  'TP Similarity': 99.80581665039062,
  'Others Similarity Average': 0.9710116410627961,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'JesusCotes',
  'Size': 27713,
  'Width': 450.0,
  'Height': 800.0,
  'TP Similarity': 99.90042114257812,
  'Others Similarity Average': 1.2472576502710582,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'JesusCotes',
  'Size': 9490,
  'Width': 225.0,
  'Height': 400.0,
  'TP Similarity': 99.94185638427734,
  'Others Similarity Average': 1.6571623094379901,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'JesusCotes',
  'Size': 3594,
  'Width': 112.5,
  'Height': 200.0,
  'TP Similarity': 99.9583740234375,
  'Others Similarity Average': 1.1920555559918284,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'JoseVilla',
  'Size': 399377,
  'Width': 1815.0,
  'Height': 2420.0,
  'TP Similarity': 91.1860580444336,
  'Others Similarity Average': 2.346254600957036,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'JoseVilla',
  'Size': 98948,
  'Width': 907.5,
  'Height': 1210.0,
  'TP Similarity': 90.84991455078125,
  'Others Similarity Average': 2.503036204725504,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'JoseVilla',
  'Size': 27354,
  'Width': 453.75,
  'Height': 605.0,
  'TP Similarity': 89.8235855102539,
  'Others Similarity Average': 1.9353933848440648,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'JoseVilla',
  'Size': 8947,
  'Width': 226.875,
  'Height': 302.5,
  'TP Similarity': 92.75220489501953,
  'Others Similarity Average': 2.669830871373415,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'JoseVilla',
  'Size': 3451,
  'Width': 113.4375,
  'Height': 151.25,
  'TP Similarity': 92.70247650146484,
  'Others Similarity Average': 4.18363743275404,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'JuanMolina',
  'Size': 389259,
  'Width': 2448.0,
  'Height': 2448.0,
  'TP Similarity': 95.17949676513672,
  'Others Similarity Average': 1.0633814785629512,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'JuanMolina',
  'Size': 81553,
  'Width': 1224.0,
  'Height': 1224.0,
  'TP Similarity': 96.65402221679688,
  'Others Similarity Average': 1.1682773265987634,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'JuanMolina',
  'Size': 23395,
  'Width': 612.0,
  'Height': 612.0,
  'TP Similarity': 95.01034545898438,
  'Others Similarity Average': 1.0215755017474293,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'JuanMolina',
  'Size': 8089,
  'Width': 306.0,
  'Height': 306.0,
  'TP Similarity': 97.44554901123047,
  'Others Similarity Average': 1.2947888098657132,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'JuanMolina',
  'Size': 3211,
  'Width': 153.0,
  'Height': 153.0,
  'TP Similarity': 98.71003723144531,
  'Others Similarity Average': 1.9645999304950237,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'ManuelRangel',
  'Size': 32563,
  'Width': 709.0,
  'Height': 945.0,
  'TP Similarity': 99.98037719726562,
  'Others Similarity Average': 5.576162705570459,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'ManuelRangel',
  'Size': 12031,
  'Width': 354.5,
  'Height': 472.5,
  'TP Similarity': 99.9818115234375,
  'Others Similarity Average': 6.385712131857872,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'ManuelRangel',
  'Size': 4864,
  'Width': 177.25,
  'Height': 236.25,
  'TP Similarity': 99.97650146484375,
  'Others Similarity Average': 5.655570055544376,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'ManuelRangel',
  'Size': 2237,
  'Width': 88.625,
  'Height': 118.125,
  'TP Similarity': 99.96275329589844,
  'Others Similarity Average': 8.10872424840927,
  'TP': 1,
  'TN': 19,
  'FP': 1,
  'FN': 0},
 {'Name': 'MariaBerdugo',
  'Size': 62694,
  'Width': 682.0,
  'Height': 910.0,
  'TP Similarity': 66.96556854248047,
  'Others Similarity Average': 2.4237869784235953,
  'TP': 0,
  'TN': 20,
  'FP': 0,
  'FN': 1},
 {'Name': 'MariaBerdugo',
  'Size': 17758,
  'Width': 341.0,
  'Height': 455.0,
  'TP Similarity': 68.5627212524414,
  'Others Similarity Average': 2.3593099430203437,
  'TP': 0,
  'TN': 20,
  'FP': 0,
  'FN': 1},
 {'Name': 'MariaBerdugo',
  'Size': 6236,
  'Width': 170.5,
  'Height': 227.5,
  'TP Similarity': 74.58274841308594,
  'Others Similarity Average': 2.2819177538156508,
  'TP': 0,
  'TN': 20,
  'FP': 0,
  'FN': 1},
 {'Name': 'MariaBerdugo',
  'Size': 2623,
  'Width': 85.25,
  'Height': 113.75,
  'TP Similarity': 85.31829071044922,
  'Others Similarity Average': 2.3524315200746058,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'MariaLopez',
  'Size': 415432,
  'Width': 2640.0,
  'Height': 1980.0,
  'TP Similarity': 96.71992492675781,
  'Others Similarity Average': 0.7637720350176096,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'MariaLopez',
  'Size': 112863,
  'Width': 1320.0,
  'Height': 990.0,
  'TP Similarity': 96.7176284790039,
  'Others Similarity Average': 0.6892186615616083,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'MariaLopez',
  'Size': 31233,
  'Width': 660.0,
  'Height': 495.0,
  'TP Similarity': 96.27423095703125,
  'Others Similarity Average': 0.6478189330548048,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'MariaLopez',
  'Size': 10439,
  'Width': 330.0,
  'Height': 247.5,
  'TP Similarity': 98.72834777832031,
  'Others Similarity Average': 1.085353945568204,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'MariaLopez',
  'Size': 4091,
  'Width': 165.0,
  'Height': 123.75,
  'TP Similarity': 99.5151596069336,
  'Others Similarity Average': 0.9405330501496791,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'MauricioDeLaHoz',
  'Size': 177074,
  'Width': 1200.0,
  'Height': 1600.0,
  'TP Similarity': 36.3138542175293,
  'Others Similarity Average': 1.2259764846414327,
  'TP': 0,
  'TN': 20,
  'FP': 0,
  'FN': 1},
 {'Name': 'MauricioDeLaHoz',
  'Size': 55242,
  'Width': 600.0,
  'Height': 800.0,
  'TP Similarity': 36.81185531616211,
  'Others Similarity Average': 1.2170237805694342,
  'TP': 0,
  'TN': 20,
  'FP': 0,
  'FN': 1},
 {'Name': 'MauricioDeLaHoz',
  'Size': 18135,
  'Width': 300.0,
  'Height': 400.0,
  'TP Similarity': 54.76950454711914,
  'Others Similarity Average': 1.6264986127614975,
  'TP': 0,
  'TN': 20,
  'FP': 0,
  'FN': 1},
 {'Name': 'MauricioDeLaHoz',
  'Size': 6269,
  'Width': 150.0,
  'Height': 200.0,
  'TP Similarity': 44.41359329223633,
  'Others Similarity Average': 1.900153711438179,
  'TP': 0,
  'TN': 20,
  'FP': 0,
  'FN': 1},
 {'Name': 'SergioFonseca',
  'Size': 362436,
  'Width': 1440.0,
  'Height': 2559.0,
  'TP Similarity': 99.69081115722656,
  'Others Similarity Average': 3.918403945490718,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'SergioFonseca',
  'Size': 94811,
  'Width': 720.0,
  'Height': 1279.5,
  'TP Similarity': 99.72557067871094,
  'Others Similarity Average': 3.44770053550601,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'SergioFonseca',
  'Size': 26284,
  'Width': 360.0,
  'Height': 639.75,
  'TP Similarity': 99.77013397216797,
  'Others Similarity Average': 3.120233351737261,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'SergioFonseca',
  'Size': 8545,
  'Width': 180.0,
  'Height': 319.875,
  'TP Similarity': 99.79846954345703,
  'Others Similarity Average': 3.3741002056747673,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'SergioFonseca',
  'Size': 3222,
  'Width': 90.0,
  'Height': 159.9375,
  'TP Similarity': 99.93584442138672,
  'Others Similarity Average': 5.828313574008644,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'YordiRochel',
  'Size': 290168,
  'Width': 1740.0,
  'Height': 2319.0,
  'TP Similarity': 96.29116821289062,
  'Others Similarity Average': 1.3835722923278808,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'YordiRochel',
  'Size': 80430,
  'Width': 870.0,
  'Height': 1159.5,
  'TP Similarity': 96.27264404296875,
  'Others Similarity Average': 1.3405101105570794,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'YordiRochel',
  'Size': 22017,
  'Width': 435.0,
  'Height': 579.75,
  'TP Similarity': 95.86953735351562,
  'Others Similarity Average': 1.6522831335663795,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'YordiRochel',
  'Size': 7461,
  'Width': 217.5,
  'Height': 289.875,
  'TP Similarity': 96.48863983154297,
  'Others Similarity Average': 2.1509024016559124,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0},
 {'Name': 'YordiRochel',
  'Size': 2889,
  'Width': 108.75,
  'Height': 144.9375,
  'TP Similarity': 97.39796447753906,
  'Others Similarity Average': 1.9979670070111752,
  'TP': 1,
  'TN': 20,
  'FP': 0,
  'FN': 0}]

 ,

[{'Name': 'AllysonSalom',
  'Size': 477613,
  'Width': 2576.0,
  'Height': 1932.0,
  'TP Similarity': 99.15478515625,
  'Others Similarity Average': 1.5014729928225279,
  'TP': 1,
  'TN': 8,
  'FP': 0,
  'FN': 0},
 {'Name': 'AllysonSalom',
  'Size': 113617,
  'Width': 1288.0,
  'Height': 966.0,
  'TP Similarity': 98.59173583984375,
  'Others Similarity Average': 1.1326271770521998,
  'TP': 1,
  'TN': 8,
  'FP': 0,
  'FN': 0},
 {'Name': 'AllysonSalom',
  'Size': 25785,
  'Width': 644.0,
  'Height': 483.0,
  'TP Similarity': 98.95576477050781,
  'Others Similarity Average': 1.0507896821945906,
  'TP': 1,
  'TN': 8,
  'FP': 0,
  'FN': 0},
 {'Name': 'AllysonSalom',
  'Size': 8292,
  'Width': 322.0,
  'Height': 241.5,
  'TP Similarity': 98.48247528076172,
  'Others Similarity Average': 1.3607451803982258,
  'TP': 1,
  'TN': 8,
  'FP': 0,
  'FN': 0},
 {'Name': 'AllysonSalom',
  'Size': 3303,
  'Width': 161.0,
  'Height': 120.75,
  'TP Similarity': 99.28514099121094,
  'Others Similarity Average': 1.3114560982212424,
  'TP': 1,
  'TN': 8,
  'FP': 0,
  'FN': 0},
 {'Name': 'AngieValencia',
  'Size': 613038,
  'Width': 2944.0,
  'Height': 2208.0,
  'TP Similarity': 96.80204772949219,
  'Others Similarity Average': 0.5912975873798132,
  'TP': 1,
  'TN': 8,
  'FP': 0,
  'FN': 0},
 {'Name': 'AngieValencia',
  'Size': 141778,
  'Width': 1472.0,
  'Height': 1104.0,
  'TP Similarity': 95.83222961425781,
  'Others Similarity Average': 0.5220485609024763,
  'TP': 1,
  'TN': 8,
  'FP': 0,
  'FN': 0},
 {'Name': 'AngieValencia',
  'Size': 35929,
  'Width': 736.0,
  'Height': 552.0,
  'TP Similarity': 96.03539276123047,
  'Others Similarity Average': 0.4757036007940769,
  'TP': 1,
  'TN': 8,
  'FP': 0,
  'FN': 0},
 {'Name': 'AngieValencia',
  'Size': 10804,
  'Width': 368.0,
  'Height': 276.0,
  'TP Similarity': 97.54324340820312,
  'Others Similarity Average': 0.5286916214972734,
  'TP': 1,
  'TN': 8,
  'FP': 0,
  'FN': 0},
 {'Name': 'AngieValencia',
  'Size': 3877,
  'Width': 184.0,
  'Height': 138.0,
  'TP Similarity': 97.58198547363281,
  'Others Similarity Average': 0.44334168918430805,
  'TP': 1,
  'TN': 8,
  'FP': 0,
  'FN': 0},
 {'Name': 'DanielDelgado',
  'Size': 558871,
  'Width': 1815.0,
  'Height': 2420.0,
  'TP Similarity': 99.99815368652344,
  'Others Similarity Average': 2.326705366373062,
  'TP': 1,
  'TN': 8,
  'FP': 0,
  'FN': 0},
 {'Name': 'DanielDelgado',
  'Size': 143024,
  'Width': 907.5,
  'Height': 1210.0,
  'TP Similarity': 99.998291015625,
  'Others Similarity Average': 2.2837339602410793,
  'TP': 1,
  'TN': 8,
  'FP': 0,
  'FN': 0},
 {'Name': 'DanielDelgado',
  'Size': 37384,
  'Width': 453.75,
  'Height': 605.0,
  'TP Similarity': 99.99884033203125,
  'Others Similarity Average': 2.248343735933304,
  'TP': 1,
  'TN': 8,
  'FP': 0,
  'FN': 0},
 {'Name': 'DanielDelgado',
  'Size': 11774,
  'Width': 226.875,
  'Height': 302.5,
  'TP Similarity': 99.9984359741211,
  'Others Similarity Average': 2.512237586081028,
  'TP': 1,
  'TN': 8,
  'FP': 0,
  'FN': 0},
 {'Name': 'DanielDelgado',
  'Size': 4364,
  'Width': 113.4375,
  'Height': 151.25,
  'TP Similarity': 99.99683380126953,
  'Others Similarity Average': 4.519347410649061,
  'TP': 1,
  'TN': 8,
  'FP': 0,
  'FN': 0},
 {'Name': 'DiegoGomez',
  'Size': 2733,
  'Width': 120.0,
  'Height': 120.0,
  'TP Similarity': 99.95091247558594,
  'Others Similarity Average': 1.2655978063121438,
  'TP': 1,
  'TN': 8,
  'FP': 0,
  'FN': 0},
 {'Name': 'DylanDeOro',
  'Size': 333912,
  'Width': 2598.0,
  'Height': 3464.0,
  'TP Similarity': 99.99186706542969,
  'Others Similarity Average': 4.483760043978691,
  'TP': 1,
  'TN': 8,
  'FP': 0,
  'FN': 0},
 {'Name': 'DylanDeOro',
  'Size': 104119,
  'Width': 1299.0,
  'Height': 1732.0,
  'TP Similarity': 99.99150085449219,
  'Others Similarity Average': 4.287475693970919,
  'TP': 1,
  'TN': 8,
  'FP': 0,
  'FN': 0},
 {'Name': 'DylanDeOro',
  'Size': 31344,
  'Width': 649.5,
  'Height': 866.0,
  'TP Similarity': 99.99459838867188,
  'Others Similarity Average': 4.276240784674883,
  'TP': 1,
  'TN': 8,
  'FP': 0,
  'FN': 0},
 {'Name': 'DylanDeOro',
  'Size': 10726,
  'Width': 324.75,
  'Height': 433.0,
  'TP Similarity': 99.98844909667969,
  'Others Similarity Average': 4.139995079487562,
  'TP': 1,
  'TN': 8,
  'FP': 0,
  'FN': 0},
 {'Name': 'DylanDeOro',
  'Size': 4321,
  'Width': 162.375,
  'Height': 216.5,
  'TP Similarity': 99.97911834716797,
  'Others Similarity Average': 4.58212136849761,
  'TP': 1,
  'TN': 8,
  'FP': 0,
  'FN': 0},
 {'Name': 'DylanDeOro',
  'Size': 2014,
  'Width': 81.1875,
  'Height': 108.25,
  'TP Similarity': 99.9062271118164,
  'Others Similarity Average': 7.302908055484295,
  'TP': 1,
  'TN': 8,
  'FP': 0,
  'FN': 0},
 {'Name': 'FelipeOspino',
  'Size': 425138,
  'Width': 2576.0,
  'Height': 1932.0,
  'TP Similarity': 99.7753677368164,
  'Others Similarity Average': 4.3533663004636765,
  'TP': 1,
  'TN': 8,
  'FP': 0,
  'FN': 0},
 {'Name': 'FelipeOspino',
  'Size': 107233,
  'Width': 1288.0,
  'Height': 966.0,
  'TP Similarity': 99.7181625366211,
  'Others Similarity Average': 3.866518124938011,
  'TP': 1,
  'TN': 8,
  'FP': 0,
  'FN': 0},
 {'Name': 'FelipeOspino',
  'Size': 29512,
  'Width': 644.0,
  'Height': 483.0,
  'TP Similarity': 99.79268646240234,
  'Others Similarity Average': 3.6183345317840576,
  'TP': 1,
  'TN': 8,
  'FP': 0,
  'FN': 0},
 {'Name': 'FelipeOspino',
  'Size': 9023,
  'Width': 322.0,
  'Height': 241.5,
  'TP Similarity': 99.91754150390625,
  'Others Similarity Average': 4.057273209095001,
  'TP': 1,
  'TN': 8,
  'FP': 0,
  'FN': 0},
 {'Name': 'FelipeOspino',
  'Size': 3281,
  'Width': 161.0,
  'Height': 120.75,
  'TP Similarity': 99.89177703857422,
  'Others Similarity Average': 3.0129978954792023,
  'TP': 1,
  'TN': 8,
  'FP': 0,
  'FN': 0},
 {'Name': 'HabidAbdala',
  'Size': 809205,
  'Width': 3392.0,
  'Height': 2544.0,
  'TP Similarity': 82.11431121826172,
  'Others Similarity Average': 1.6522499360144138,
  'TP': 1,
  'TN': 8,
  'FP': 0,
  'FN': 0},
 {'Name': 'HabidAbdala',
  'Size': 156232,
  'Width': 1696.0,
  'Height': 1272.0,
  'TP Similarity': 75.71408081054688,
  'Others Similarity Average': 1.4315274469554424,
  'TP': 0,
  'TN': 8,
  'FP': 0,
  'FN': 1},
 {'Name': 'HabidAbdala',
  'Size': 40656,
  'Width': 848.0,
  'Height': 636.0,
  'TP Similarity': 79.81193542480469,
  'Others Similarity Average': 1.4778800811618567,
  'TP': 0,
  'TN': 8,
  'FP': 0,
  'FN': 1},
 {'Name': 'HabidAbdala',
  'Size': 12674,
  'Width': 424.0,
  'Height': 318.0,
  'TP Similarity': 85.28485107421875,
  'Others Similarity Average': 1.5141345411539078,
  'TP': 1,
  'TN': 8,
  'FP': 0,
  'FN': 0},
 {'Name': 'HabidAbdala',
  'Size': 4710,
  'Width': 212.0,
  'Height': 159.0,
  'TP Similarity': 90.70542907714844,
  'Others Similarity Average': 1.5198151860386133,
  'TP': 1,
  'TN': 8,
  'FP': 0,
  'FN': 0},
 {'Name': 'JuanLozano',
  'Size': 130563,
  'Width': 1179.0,
  'Height': 1329.0,
  'TP Similarity': 99.98599243164062,
  'Others Similarity Average': 1.3044462651014328,
  'TP': 1,
  'TN': 8,
  'FP': 0,
  'FN': 0},
 {'Name': 'JuanLozano',
  'Size': 45196,
  'Width': 589.5,
  'Height': 664.5,
  'TP Similarity': 99.98872375488281,
  'Others Similarity Average': 1.4511742740869522,
  'TP': 1,
  'TN': 8,
  'FP': 0,
  'FN': 0},
 {'Name': 'JuanLozano',
  'Size': 14577,
  'Width': 294.75,
  'Height': 332.25,
  'TP Similarity': 99.98878479003906,
  'Others Similarity Average': 1.171222910284996,
  'TP': 1,
  'TN': 8,
  'FP': 0,
  'FN': 0},
 {'Name': 'JuanLozano',
  'Size': 4718,
  'Width': 147.375,
  'Height': 166.125,
  'TP Similarity': 99.9868392944336,
  'Others Similarity Average': 1.0660089887678623,
  'TP': 1,
  'TN': 8,
  'FP': 0,
  'FN': 0},
 {'Name': 'WilmanDaza',
  'Size': 700425,
  'Width': 3088.0,
  'Height': 2316.0,
  'TP Similarity': 99.8836669921875,
  'Others Similarity Average': 1.2580478861927986,
  'TP': 1,
  'TN': 8,
  'FP': 0,
  'FN': 0},
 {'Name': 'WilmanDaza',
  'Size': 150713,
  'Width': 1544.0,
  'Height': 1158.0,
  'TP Similarity': 99.9389877319336,
  'Others Similarity Average': 1.3852234100922942,
  'TP': 1,
  'TN': 8,
  'FP': 0,
  'FN': 0},
 {'Name': 'WilmanDaza',
  'Size': 39575,
  'Width': 772.0,
  'Height': 579.0,
  'TP Similarity': 99.90797424316406,
  'Others Similarity Average': 1.232069575227797,
  'TP': 1,
  'TN': 8,
  'FP': 0,
  'FN': 0},
 {'Name': 'WilmanDaza',
  'Size': 11693,
  'Width': 386.0,
  'Height': 289.5,
  'TP Similarity': 99.88516235351562,
  'Others Similarity Average': 1.6435400387272239,
  'TP': 1,
  'TN': 8,
  'FP': 0,
  'FN': 0},
 {'Name': 'WilmanDaza',
  'Size': 4041,
  'Width': 193.0,
  'Height': 144.75,
  'TP Similarity': 99.8469009399414,
  'Others Similarity Average': 1.3035837737843394,
  'TP': 1,
  'TN': 8,
  'FP': 0,
  'FN': 0}]

,

[{'Name': 'AliRada',
  'Size': 423604,
  'Width': 1448.0,
  'Height': 2573.0,
  'TP Similarity': 37.42267990112305,
  'Others Similarity Average': 2.0274264716050205,
  'TP': 0,
  'TN': 17,
  'FP': 0,
  'FN': 1},
 {'Name': 'AliRada',
  'Size': 119566,
  'Width': 724.0,
  'Height': 1286.5,
  'TP Similarity': 32.34839630126953,
  'Others Similarity Average': 2.1045171250315273,
  'TP': 0,
  'TN': 17,
  'FP': 0,
  'FN': 1},
 {'Name': 'AliRada',
  'Size': 29539,
  'Width': 362.0,
  'Height': 643.25,
  'TP Similarity': 43.17999267578125,
  'Others Similarity Average': 2.2137215908835914,
  'TP': 0,
  'TN': 17,
  'FP': 0,
  'FN': 1},
 {'Name': 'AliRada',
  'Size': 10022,
  'Width': 181.0,
  'Height': 321.625,
  'TP Similarity': 50.519187927246094,
  'Others Similarity Average': 2.686490973128992,
  'TP': 0,
  'TN': 17,
  'FP': 0,
  'FN': 1},
 {'Name': 'AliRada',
  'Size': 3555,
  'Width': 90.5,
  'Height': 160.8125,
  'TP Similarity': 45.54950714111328,
  'Others Similarity Average': 2.7257701833458508,
  'TP': 0,
  'TN': 17,
  'FP': 0,
  'FN': 1},
 {'Name': 'AndreaQuintero',
  'Size': 575219,
  'Width': 1815.0,
  'Height': 2420.0,
  'TP Similarity': 90.9809341430664,
  'Others Similarity Average': 0.8814706002526423,
  'TP': 1,
  'TN': 17,
  'FP': 0,
  'FN': 0},
 {'Name': 'AndreaQuintero',
  'Size': 144314,
  'Width': 907.5,
  'Height': 1210.0,
  'TP Similarity': 89.9306411743164,
  'Others Similarity Average': 0.7733116695547805,
  'TP': 1,
  'TN': 17,
  'FP': 0,
  'FN': 0},
 {'Name': 'AndreaQuintero',
  'Size': 40325,
  'Width': 453.75,
  'Height': 605.0,
  'TP Similarity': 92.51496887207031,
  'Others Similarity Average': 0.71064443141222,
  'TP': 1,
  'TN': 17,
  'FP': 0,
  'FN': 0},
 {'Name': 'AndreaQuintero',
  'Size': 12612,
  'Width': 226.875,
  'Height': 302.5,
  'TP Similarity': 96.76468658447266,
  'Others Similarity Average': 0.8376237658016822,
  'TP': 1,
  'TN': 17,
  'FP': 0,
  'FN': 0},
 {'Name': 'AndreaQuintero',
  'Size': 4529,
  'Width': 113.4375,
  'Height': 151.25,
  'TP Similarity': 97.84297180175781,
  'Others Similarity Average': 0.7440004541593439,
  'TP': 1,
  'TN': 17,
  'FP': 0,
  'FN': 0},
 {'Name': 'AndresFabregas',
  'Size': 803841,
  'Width': 3088.0,
  'Height': 2316.0,
  'TP Similarity': 98.87459564208984,
  'Others Similarity Average': 3.03892352081397,
  'TP': 1,
  'TN': 17,
  'FP': 0,
  'FN': 0},
 {'Name': 'AndresFabregas',
  'Size': 194039,
  'Width': 1544.0,
  'Height': 1158.0,
  'TP Similarity': 99.0431137084961,
  'Others Similarity Average': 3.383982569855802,
  'TP': 1,
  'TN': 17,
  'FP': 0,
  'FN': 0},
 {'Name': 'AndresFabregas',
  'Size': 53778,
  'Width': 772.0,
  'Height': 579.0,
  'TP Similarity': 99.25608825683594,
  'Others Similarity Average': 3.0552724950453816,
  'TP': 1,
  'TN': 17,
  'FP': 0,
  'FN': 0},
 {'Name': 'AndresFabregas',
  'Size': 17793,
  'Width': 386.0,
  'Height': 289.5,
  'TP Similarity': 99.52821350097656,
  'Others Similarity Average': 2.7275342844864903,
  'TP': 1,
  'TN': 17,
  'FP': 0,
  'FN': 0},
 {'Name': 'AndresFabregas',
  'Size': 6474,
  'Width': 193.0,
  'Height': 144.75,
  'TP Similarity': 99.68233489990234,
  'Others Similarity Average': 2.836254746598356,
  'TP': 1,
  'TN': 17,
  'FP': 0,
  'FN': 0},
 {'Name': 'AndresNarvaez',
  'Size': 1027806,
  'Width': 4032.0,
  'Height': 3024.0,
  'TP Similarity': 98.43746948242188,
  'Others Similarity Average': 1.7416104437673794,
  'TP': 1,
  'TN': 17,
  'FP': 0,
  'FN': 0},
 {'Name': 'AndresNarvaez',
  'Size': 326851,
  'Width': 2016.0,
  'Height': 1512.0,
  'TP Similarity': 98.82810974121094,
  'Others Similarity Average': 1.504924065049957,
  'TP': 1,
  'TN': 17,
  'FP': 0,
  'FN': 0},
 {'Name': 'AndresNarvaez',
  'Size': 92779,
  'Width': 1008.0,
  'Height': 756.0,
  'TP Similarity': 98.87079620361328,
  'Others Similarity Average': 1.5038588739493315,
  'TP': 1,
  'TN': 17,
  'FP': 0,
  'FN': 0},
 {'Name': 'AndresNarvaez',
  'Size': 26184,
  'Width': 504.0,
  'Height': 378.0,
  'TP Similarity': 98.70446014404297,
  'Others Similarity Average': 1.4514068671885658,
  'TP': 1,
  'TN': 17,
  'FP': 0,
  'FN': 0},
 {'Name': 'AndresNarvaez',
  'Size': 8256,
  'Width': 252.0,
  'Height': 189.0,
  'TP Similarity': 98.7273941040039,
  'Others Similarity Average': 1.6470334065311096,
  'TP': 1,
  'TN': 17,
  'FP': 0,
  'FN': 0},
 {'Name': 'AndresNarvaez',
  'Size': 3127,
  'Width': 126.0,
  'Height': 94.5,
  'TP Similarity': 98.08769226074219,
  'Others Similarity Average': 1.843891960733077,
  'TP': 1,
  'TN': 17,
  'FP': 0,
  'FN': 0},
 {'Name': 'BrayanRubiano',
  'Size': 803597,
  'Width': 3120.0,
  'Height': 4208.0,
  'TP Similarity': 95.56987762451172,
  'Others Similarity Average': 1.2388239814078106,
  'TP': 1,
  'TN': 17,
  'FP': 0,
  'FN': 0},
 {'Name': 'BrayanRubiano',
  'Size': 173727,
  'Width': 1560.0,
  'Height': 2104.0,
  'TP Similarity': 95.35015869140625,
  'Others Similarity Average': 1.2958925283130478,
  'TP': 1,
  'TN': 17,
  'FP': 0,
  'FN': 0},
 {'Name': 'BrayanRubiano',
  'Size': 52701,
  'Width': 780.0,
  'Height': 1052.0,
  'TP Similarity': 95.82097625732422,
  'Others Similarity Average': 1.322022065958556,
  'TP': 1,
  'TN': 17,
  'FP': 0,
  'FN': 0},
 {'Name': 'BrayanRubiano',
  'Size': 16655,
  'Width': 390.0,
  'Height': 526.0,
  'TP Similarity': 94.2302017211914,
  'Others Similarity Average': 1.2745090157670134,
  'TP': 1,
  'TN': 17,
  'FP': 0,
  'FN': 0},
 {'Name': 'BrayanRubiano',
  'Size': 5996,
  'Width': 195.0,
  'Height': 263.0,
  'TP Similarity': 95.5455551147461,
  'Others Similarity Average': 1.2707691232071203,
  'TP': 1,
  'TN': 17,
  'FP': 0,
  'FN': 0},
 {'Name': 'BrayanRubiano',
  'Size': 2621,
  'Width': 97.5,
  'Height': 131.5,
  'TP Similarity': 97.15536499023438,
  'Others Similarity Average': 1.3741535373470362,
  'TP': 1,
  'TN': 17,
  'FP': 0,
  'FN': 0},
 {'Name': 'DiegoGomez',
  'Size': 2733,
  'Width': 120.0,
  'Height': 120.0,
  'TP Similarity': 99.26193237304688,
  'Others Similarity Average': 1.7082959654576637,
  'TP': 1,
  'TN': 17,
  'FP': 0,
  'FN': 0},
 {'Name': 'EdwinUtria',
  'Size': 1430251,
  'Width': 5184.0,
  'Height': 3880.0,
  'TP Similarity': 98.96647644042969,
  'Others Similarity Average': 2.3324328818741966,
  'TP': 1,
  'TN': 17,
  'FP': 0,
  'FN': 0},
 {'Name': 'EdwinUtria',
  'Size': 420201,
  'Width': 2592.0,
  'Height': 1940.0,
  'TP Similarity': 99.11621856689453,
  'Others Similarity Average': 2.3346503306837643,
  'TP': 1,
  'TN': 17,
  'FP': 0,
  'FN': 0},
 {'Name': 'EdwinUtria',
  'Size': 100005,
  'Width': 1296.0,
  'Height': 970.0,
  'TP Similarity': 98.16593170166016,
  'Others Similarity Average': 2.2949986221159207,
  'TP': 1,
  'TN': 17,
  'FP': 0,
  'FN': 0},
 {'Name': 'EdwinUtria',
  'Size': 25986,
  'Width': 648.0,
  'Height': 485.0,
  'TP Similarity': 98.65901947021484,
  'Others Similarity Average': 2.0184524953365326,
  'TP': 1,
  'TN': 17,
  'FP': 0,
  'FN': 0},
 {'Name': 'EdwinUtria',
  'Size': 8600,
  'Width': 324.0,
  'Height': 242.5,
  'TP Similarity': 98.96392822265625,
  'Others Similarity Average': 2.802630364894867,
  'TP': 1,
  'TN': 17,
  'FP': 0,
  'FN': 0},
 {'Name': 'EdwinUtria',
  'Size': 3543,
  'Width': 162.0,
  'Height': 121.25,
  'TP Similarity': 99.24125671386719,
  'Others Similarity Average': 3.055847359054229,
  'TP': 1,
  'TN': 17,
  'FP': 0,
  'FN': 0},
 {'Name': 'GabrielaBecerra',
  'Size': 793418,
  'Width': 3088.0,
  'Height': 2316.0,
  'TP Similarity': 49.96076202392578,
  'Others Similarity Average': 1.694888133336516,
  'TP': 0,
  'TN': 17,
  'FP': 0,
  'FN': 1},
 {'Name': 'GabrielaBecerra',
  'Size': 195763,
  'Width': 1544.0,
  'Height': 1158.0,
  'TP Similarity': 59.83797836303711,
  'Others Similarity Average': 1.5710788351647995,
  'TP': 0,
  'TN': 17,
  'FP': 0,
  'FN': 1},
 {'Name': 'GabrielaBecerra',
  'Size': 57448,
  'Width': 772.0,
  'Height': 579.0,
  'TP Similarity': 53.558528900146484,
  'Others Similarity Average': 1.48492651914849,
  'TP': 0,
  'TN': 17,
  'FP': 0,
  'FN': 1},
 {'Name': 'GabrielaBecerra',
  'Size': 18310,
  'Width': 386.0,
  'Height': 289.5,
  'TP Similarity': 61.0822868347168,
  'Others Similarity Average': 1.7194976640098236,
  'TP': 0,
  'TN': 17,
  'FP': 0,
  'FN': 1},
 {'Name': 'GabrielaBecerra',
  'Size': 6270,
  'Width': 193.0,
  'Height': 144.75,
  'TP Similarity': 79.22442626953125,
  'Others Similarity Average': 1.577272606684881,
  'TP': 0,
  'TN': 17,
  'FP': 0,
  'FN': 1},
 {'Name': 'JuanHernandez',
  'Size': 371420,
  'Width': 1816.0,
  'Height': 2420.0,
  'TP Similarity': 97.93284606933594,
  'Others Similarity Average': 1.2475979832165383,
  'TP': 1,
  'TN': 17,
  'FP': 0,
  'FN': 0},
 {'Name': 'JuanHernandez',
  'Size': 101855,
  'Width': 908.0,
  'Height': 1210.0,
  'TP Similarity': 97.22783660888672,
  'Others Similarity Average': 1.2730933837592602,
  'TP': 1,
  'TN': 17,
  'FP': 0,
  'FN': 0},
 {'Name': 'JuanHernandez',
  'Size': 31855,
  'Width': 454.0,
  'Height': 605.0,
  'TP Similarity': 97.60566711425781,
  'Others Similarity Average': 1.2474055566331919,
  'TP': 1,
  'TN': 17,
  'FP': 0,
  'FN': 0},
 {'Name': 'JuanHernandez',
  'Size': 11013,
  'Width': 227.0,
  'Height': 302.5,
  'TP Similarity': 96.56436920166016,
  'Others Similarity Average': 1.775580350528745,
  'TP': 1,
  'TN': 17,
  'FP': 0,
  'FN': 0},
 {'Name': 'JuanHernandez',
  'Size': 3904,
  'Width': 113.5,
  'Height': 151.25,
  'TP Similarity': 97.2164306640625,
  'Others Similarity Average': 2.269962064921856,
  'TP': 1,
  'TN': 17,
  'FP': 0,
  'FN': 0},
 {'Name': 'JuanQuintero',
  'Size': 604539,
  'Width': 3000.0,
  'Height': 3000.0,
  'TP Similarity': 98.88557434082031,
  'Others Similarity Average': 1.7413329929113388,
  'TP': 1,
  'TN': 17,
  'FP': 0,
  'FN': 0},
 {'Name': 'JuanQuintero',
  'Size': 202222,
  'Width': 1500.0,
  'Height': 1500.0,
  'TP Similarity': 99.02706146240234,
  'Others Similarity Average': 1.712721610770506,
  'TP': 1,
  'TN': 17,
  'FP': 0,
  'FN': 0},
 {'Name': 'JuanQuintero',
  'Size': 72826,
  'Width': 750.0,
  'Height': 750.0,
  'TP Similarity': 98.84668731689453,
  'Others Similarity Average': 1.8288570743273287,
  'TP': 1,
  'TN': 17,
  'FP': 0,
  'FN': 0},
 {'Name': 'JuanQuintero',
  'Size': 24727,
  'Width': 375.0,
  'Height': 375.0,
  'TP Similarity': 98.71051788330078,
  'Others Similarity Average': 2.057279645520098,
  'TP': 1,
  'TN': 17,
  'FP': 0,
  'FN': 0},
 {'Name': 'JuanQuintero',
  'Size': 8260,
  'Width': 187.5,
  'Height': 187.5,
  'TP Similarity': 92.8564453125,
  'Others Similarity Average': 1.8372801682528328,
  'TP': 1,
  'TN': 17,
  'FP': 0,
  'FN': 0},
 {'Name': 'JuanQuintero',
  'Size': 2804,
  'Width': 93.75,
  'Height': 93.75,
  'TP Similarity': 39.953975677490234,
  'Others Similarity Average': 1.7611178887241028,
  'TP': 0,
  'TN': 17,
  'FP': 0,
  'FN': 1},
 {'Name': 'JuanSanchez',
  'Size': 486997,
  'Width': 3088.0,
  'Height': 2320.0,
  'TP Similarity': 99.99524688720703,
  'Others Similarity Average': 1.903458717991324,
  'TP': 1,
  'TN': 17,
  'FP': 0,
  'FN': 0},
 {'Name': 'JuanSanchez',
  'Size': 135707,
  'Width': 1544.0,
  'Height': 1160.0,
  'TP Similarity': 99.99344635009766,
  'Others Similarity Average': 1.931849669884233,
  'TP': 1,
  'TN': 17,
  'FP': 0,
  'FN': 0},
 {'Name': 'JuanSanchez',
  'Size': 38431,
  'Width': 772.0,
  'Height': 580.0,
  'TP Similarity': 99.9958267211914,
  'Others Similarity Average': 2.018664879833951,
  'TP': 1,
  'TN': 17,
  'FP': 0,
  'FN': 0},
 {'Name': 'JuanSanchez',
  'Size': 12446,
  'Width': 386.0,
  'Height': 290.0,
  'TP Similarity': 99.9948501586914,
  'Others Similarity Average': 2.353536958203596,
  'TP': 1,
  'TN': 17,
  'FP': 0,
  'FN': 0},
 {'Name': 'JuanSanchez',
  'Size': 4577,
  'Width': 193.0,
  'Height': 145.0,
  'TP Similarity': 99.99451446533203,
  'Others Similarity Average': 2.569750742000692,
  'TP': 1,
  'TN': 17,
  'FP': 0,
  'FN': 0},
 {'Name': 'LucianaDeLaRosa',
  'Size': 872301,
  'Width': 3088.0,
  'Height': 2316.0,
  'TP Similarity': 70.83712768554688,
  'Others Similarity Average': 0.7655029820606989,
  'TP': 0,
  'TN': 17,
  'FP': 0,
  'FN': 1},
 {'Name': 'LucianaDeLaRosa',
  'Size': 211733,
  'Width': 1544.0,
  'Height': 1158.0,
  'TP Similarity': 59.37588119506836,
  'Others Similarity Average': 0.5905034038073876,
  'TP': 0,
  'TN': 17,
  'FP': 0,
  'FN': 1},
 {'Name': 'LucianaDeLaRosa',
  'Size': 60380,
  'Width': 772.0,
  'Height': 579.0,
  'TP Similarity': 59.10051345825195,
  'Others Similarity Average': 0.6689146912711508,
  'TP': 0,
  'TN': 17,
  'FP': 0,
  'FN': 1},
 {'Name': 'LucianaDeLaRosa',
  'Size': 17970,
  'Width': 386.0,
  'Height': 289.5,
  'TP Similarity': 68.53214263916016,
  'Others Similarity Average': 0.9406428284504834,
  'TP': 0,
  'TN': 17,
  'FP': 0,
  'FN': 1},
 {'Name': 'LucianaDeLaRosa',
  'Size': 6023,
  'Width': 193.0,
  'Height': 144.75,
  'TP Similarity': 87.51933288574219,
  'Others Similarity Average': 1.065165660398848,
  'TP': 1,
  'TN': 17,
  'FP': 0,
  'FN': 0},
 {'Name': 'MariaFerrer',
  'Size': 893390,
  'Width': 3264.0,
  'Height': 2448.0,
  'TP Similarity': 95.52566528320312,
  'Others Similarity Average': 1.564881468520445,
  'TP': 1,
  'TN': 17,
  'FP': 0,
  'FN': 0},
 {'Name': 'MariaFerrer',
  'Size': 166789,
  'Width': 1632.0,
  'Height': 1224.0,
  'TP Similarity': 93.31851196289062,
  'Others Similarity Average': 1.5689724429565317,
  'TP': 1,
  'TN': 17,
  'FP': 0,
  'FN': 0},
 {'Name': 'MariaFerrer',
  'Size': 46608,
  'Width': 816.0,
  'Height': 612.0,
  'TP Similarity': 92.63092041015625,
  'Others Similarity Average': 1.4654040354139664,
  'TP': 1,
  'TN': 17,
  'FP': 0,
  'FN': 0},
 {'Name': 'MariaFerrer',
  'Size': 14621,
  'Width': 408.0,
  'Height': 306.0,
  'TP Similarity': 93.80052947998047,
  'Others Similarity Average': 1.726025691803764,
  'TP': 1,
  'TN': 17,
  'FP': 0,
  'FN': 0},
 {'Name': 'MariaFerrer',
  'Size': 5219,
  'Width': 204.0,
  'Height': 153.0,
  'TP Similarity': 94.43183898925781,
  'Others Similarity Average': 1.5416566186091478,
  'TP': 1,
  'TN': 17,
  'FP': 0,
  'FN': 0},
 {'Name': 'MayraCarreno',
  'Size': 187182,
  'Width': 1500.0,
  'Height': 1500.0,
  'TP Similarity': 99.9745101928711,
  'Others Similarity Average': 0.7319272436639842,
  'TP': 1,
  'TN': 17,
  'FP': 0,
  'FN': 0},
 {'Name': 'MayraCarreno',
  'Size': 74337,
  'Width': 750.0,
  'Height': 750.0,
  'TP Similarity': 99.95807647705078,
  'Others Similarity Average': 0.9324634136522517,
  'TP': 1,
  'TN': 17,
  'FP': 0,
  'FN': 0},
 {'Name': 'MayraCarreno',
  'Size': 26005,
  'Width': 375.0,
  'Height': 375.0,
  'TP Similarity': 99.93202209472656,
  'Others Similarity Average': 1.7527234712067772,
  'TP': 1,
  'TN': 17,
  'FP': 0,
  'FN': 0},
 {'Name': 'MayraCarreno',
  'Size': 8892,
  'Width': 187.5,
  'Height': 187.5,
  'TP Similarity': 99.63957214355469,
  'Others Similarity Average': 2.727768278297256,
  'TP': 1,
  'TN': 17,
  'FP': 0,
  'FN': 0},
 {'Name': 'MayraCarreno',
  'Size': 3111,
  'Width': 93.75,
  'Height': 93.75,
  'TP Similarity': 44.75886535644531,
  'Others Similarity Average': 3.3833735381417416,
  'TP': 0,
  'TN': 17,
  'FP': 0,
  'FN': 1},
 {'Name': 'SamirBarcelo',
  'Size': 117506,
  'Width': 720.0,
  'Height': 1280.0,
  'TP Similarity': 99.8543930053711,
  'Others Similarity Average': 1.506772675058421,
  'TP': 1,
  'TN': 17,
  'FP': 0,
  'FN': 0},
 {'Name': 'SamirBarcelo',
  'Size': 30294,
  'Width': 360.0,
  'Height': 640.0,
  'TP Similarity': 99.88528442382812,
  'Others Similarity Average': 1.356578478041817,
  'TP': 1,
  'TN': 17,
  'FP': 0,
  'FN': 0},
 {'Name': 'SamirBarcelo',
  'Size': 9120,
  'Width': 180.0,
  'Height': 320.0,
  'TP Similarity': 99.89087677001953,
  'Others Similarity Average': 1.4207685484605677,
  'TP': 1,
  'TN': 17,
  'FP': 0,
  'FN': 0},
 {'Name': 'SamirBarcelo',
  'Size': 3411,
  'Width': 90.0,
  'Height': 160.0,
  'TP Similarity': 99.84569549560547,
  'Others Similarity Average': 1.358024710241486,
  'TP': 1,
  'TN': 17,
  'FP': 0,
  'FN': 0},
 {'Name': 'SteevenBallena',
  'Size': 299291,
  'Width': 1448.0,
  'Height': 2573.0,
  'TP Similarity': 97.96041870117188,
  'Others Similarity Average': 3.20056434622144,
  'TP': 1,
  'TN': 17,
  'FP': 0,
  'FN': 0},
 {'Name': 'SteevenBallena',
  'Size': 78045,
  'Width': 724.0,
  'Height': 1286.5,
  'TP Similarity': 97.86393737792969,
  'Others Similarity Average': 3.343468687004026,
  'TP': 1,
  'TN': 17,
  'FP': 0,
  'FN': 0},
 {'Name': 'SteevenBallena',
  'Size': 23893,
  'Width': 362.0,
  'Height': 643.25,
  'TP Similarity': 99.40550994873047,
  'Others Similarity Average': 4.698429117307944,
  'TP': 1,
  'TN': 17,
  'FP': 0,
  'FN': 0},
 {'Name': 'SteevenBallena',
  'Size': 7818,
  'Width': 181.0,
  'Height': 321.625,
  'TP Similarity': 99.21546173095703,
  'Others Similarity Average': 4.349162814590861,
  'TP': 1,
  'TN': 17,
  'FP': 0,
  'FN': 0},
 {'Name': 'SteevenBallena',
  'Size': 2974,
  'Width': 90.5,
  'Height': 160.8125,
  'TP Similarity': 99.4993896484375,
  'Others Similarity Average': 3.7702547434498284,
  'TP': 1,
  'TN': 17,
  'FP': 0,
  'FN': 0}]

]



In [78]:
biggestResolution=0
leastResolution=99999999
numberOfElements=0

#Obtain number of elements, greatest and lowest resolution value in order to apply Larson's rule
for element in resolutionControlResults:
    for object in element:
        if(object["Size"]>biggestResolution):
            biggestResolution=object["Size"]
        if(object["Size"]<leastResolution):
            leastResolution=object["Size"]
        numberOfElements=numberOfElements+1



numberOfIntervals=math.ceil(1+3.332*np.log10(numberOfElements))
step=math.ceil((biggestResolution-leastResolution)/(numberOfIntervals))
print(f"Biggest {biggestResolution} and least {leastResolution}")
print(f"number of intervals: {numberOfIntervals}")
print(f"step : {step}")



Biggest 1430251 and least 2014
number of intervals: 10
step : 142824


In [79]:
resultsClassified=[[],[],[],[],[],[],[],[],[],[]]
for element in resolutionControlResults:
    for object in element:
        if (object["Size"]>=leastResolution and object["Size"]<(leastResolution+step)):
            resultsClassified[0].append(object)
        elif(object["Size"]>=(leastResolution+step) and object["Size"]<(leastResolution+2*step)):
            resultsClassified[1].append(object)
        elif(object["Size"]>=(leastResolution+2*step) and object["Size"]<(leastResolution+3*step)):
            resultsClassified[2].append(object)
        elif(object["Size"]>=(leastResolution+3*step) and object["Size"]<(leastResolution+4*step)):
            resultsClassified[3].append(object)
        elif(object["Size"]>=(leastResolution+4*step) and object["Size"]<(leastResolution+5*step)):
            resultsClassified[4].append(object)
        elif(object["Size"]>=(leastResolution+5*step) and object["Size"]<(leastResolution+6*step)):
            resultsClassified[5].append(object)
        elif(object["Size"]>=(leastResolution+6*step) and object["Size"]<(leastResolution+7*step)):
            resultsClassified[6].append(object)
        elif(object["Size"]>=(leastResolution+7*step) and object["Size"]<(leastResolution+8*step)):
            resultsClassified[7].append(object)
        elif(object["Size"]>=(leastResolution+8*step) and object["Size"]<(leastResolution+9*step)):
            resultsClassified[8].append(object)
        elif(object["Size"]>=(leastResolution+9*step) and object["Size"]<(leastResolution+10*step)):
            resultsClassified[9].append(object)



Brigthness control

In [39]:

image= Image.open(r"C:\Users\User\Desktop\FinalProject\ImagesRekognition\StudentsRobotics\AlejandroCasallas.jpeg")

enhancer=ImageEnhance.Brightness(image)

enhancer_2=ImageEnhance.Color(image)

enhanced_img=enhancer.enhance(0.2)
enhanced_img_2=enhancer_2.enhance(0)
enhanced_img.save("modifiedBrightness.jpg")
enhanced_img_2.save("modifiedColor.jpg")

In [ ]:
def calculate_brightness_simple(image_path):
    img = Image.open(image_path)
    gray_img = img.convert('L')
    pixel_array = np.array(gray_img)
    average_brightness = np.mean(pixel_array)
    normalized_brightness = average_brightness / 255

    return normalized_brightness


In [ ]:

roboticsFolderPath=r"C:\Users\User\Desktop\FinalProject\ImagesRekognition\StudentsRobotics"

i=0
brightness_counter=0
for image in os.listdir(roboticsFolderPath):
    image_path=rf"C:\Users\User\Desktop\FinalProject\ImagesRekognition\StudentsRobotics\{image}"
    brightness=calculate_brightness_simple(image_path)
    brightness_counter=brightness_counter+brightness
    i=i+1
    print(f"brillo de {image} es {brightness}")

print(f"Average brightness is {float(brightness_counter/i)}")


In [67]:
image=r"C:\Users\User\Desktop\FinalProject\ImagesRekognition\StudentsRobotics\ManuelRangel.jpeg"

image_file=Image.open(image)
initialBrightnessFactor=calculate_brightness_simple(image)
requiredBrightnessFactor=initialBrightnessFactor*2

print("--------")

enhancer=ImageEnhance.Brightness(image_file)

brightnessFactor=requiredBrightnessFactor/initialBrightnessFactor
enhanced_img=enhancer.enhance(brightnessFactor)
enhanced_img.save("modifiedBrightness.jpg")
calculate_brightness_simple("modifiedBrightness.jpg")



--------


0.9221159631381153